<a href="https://colab.research.google.com/github/choijihee0619/pytorch/blob/main/%EC%BD%94%EB%9E%A9%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello World')

Hello World


In [ ]:
%%time

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

CPU times: user 37 µs, sys: 0 ns, total: 37 µs
Wall time: 40.3 µs


In [ ]:
%%time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu
CPU times: user 1.52 ms, sys: 0 ns, total: 1.52 ms
Wall time: 1.71 ms


In [ ]:
%%time

cifar10_train = CIFAR10("./data", train=True, download=True, transform=transforms.ToTensor())
cifar10_test = CIFAR10("./data", train=False, download=True, transform=transforms.ToTensor())

batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=batch_size, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → (N, 16, 14, 14)
        x = self.pool(F.relu(self.conv2(x)))  # → (N, 32, 5, 5)
        x = x.view(x.size(0), -1)             # → (N, 32*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net().to(device)
loss_fnc = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

record_loss_train = []
record_loss_test = []

x_test, t_test = next(iter(test_loader))
if torch.cuda.is_available():
    x_test, t_test = x_test.cuda(), t_test.cuda()

for i in range(10):
    net.train()
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):
        if torch.cuda.is_available():
              x, t = x.cuda(), t.cuda()
        y = net(x)
        loss = loss_fnc(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)


    net.eval()
    y_test = net(x_test)
    loss_test = loss_fnc(y_test, t_test).item()
    record_loss_test.append(loss_test)


CPU times: user 6min 39s, sys: 347 ms, total: 6min 39s
Wall time: 6min 42s
